In [ ]:
import glob
import os
import math
from pathlib import Path
import sys

import numpy as np
import pandas as pd

import wandb

from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from otc.models.activation import ReGLU
from otc.models.fttransformer import (
    FeatureTokenizer,
    FTTransformer,
    Transformer,
)

from otc.data.dataset import TabDataset
from otc.data.dataloader import TabDataLoader
from otc.features.build_features import features_classical, features_classical_size
from otc.optim.early_stopping import EarlyStopping

In [ ]:
run = wandb.init(project="thesis", entity="fbv")

dataset = "fbv/thesis/ise_unsupervised_log_standardized_clipped:latest"
artifact = run.use_artifact(dataset)
data_dir = artifact.download()

In [ ]:
# preserve relative ordering, sample for testing ache
frac = 0.05

X_train = pd.read_parquet(Path(data_dir, "train_set.parquet"), engine="fastparquet")

X_train = X_train.sample(frac=frac, random_state=42)
y_train = X_train["buy_sell"] # y = 0
X_train = X_train[features_classical_size]

In [ ]:
training_data = TabDataset(X_train, y_train)
x_cont = training_data.x_cont
x_cat = training_data.x_cat

In [ ]:
def gen_perm(X):
    """
    Generate index permutation.
    """
    if X is None:
        return None
    return torch.randint_like(X, X.shape[0], dtype=torch.long)

x_cont_perm = gen_perm(x_cont)
x_cat_perm = gen_perm(x_cat)

In [ ]:
def gen_masks(X, perm, corrupt_probability = 0.15):
    """
    Generate binary mask for detection.
    """

    masks = torch.empty_like(X).bernoulli(p=corrupt_probability).bool()
    new_masks = masks & (X != X[perm, torch.arange(X.shape[1], device=X.device)])
    return new_masks

# FIXME: probably generate for train and val set
x_cont_mask = gen_masks(training_data.x_cont, x_cont_perm)

if training_data.x_cat is not None:
    x_cat_mask = gen_masks(training_data.x_cat, x_cat_perm)
else:
    x_cat_mask = None

In [ ]:
# replace at permutation
x_cont_permuted = torch.gather(x_cont, 0, x_cont_perm)

# replace at mask = True
x_cont[x_cont_mask] = x_cont_permuted[x_cont_mask]

if x_cat is not None:

    # along the 0 axis get elements based on perm_cat
    x_cat_permuted = torch.gather(x_cat, 0, x_cat_perm)
    
    # replace at mask
    x_cat[x_cat_mask] = x_cat_permuted[x_cat_mask]

In [ ]:
# merge masks [mask_num, mask_cat]
if x_cat != None:
    masks = torch.cat([x_cont_mask, x_cat_mask], dim=1)
else:
    masks = x_cont_mask

In [ ]:
device = "cuda"
batch_size = 16192

dl_params = {
    "batch_size": batch_size,
    "shuffle": False,
    "device": device,
}

train_loader = TabDataLoader(
    x_cat,
    x_cont,
    masks, 
    **dl_params
)

## Runfield 🛫

In [ ]:
class CLSHead(nn.Module):
    """
    2 Layer MLP projection head
    """
    # d_in is last dim of transformer output torch.Size([4, 17, 96]) -> 96 (ok)
    # d_out -> last dim of output here [4,17,1] -> [4, 16] (ok)
    def __init__(self, *, d_in: int, d_hidden: int):
        super().__init__()
        self.first = nn.Linear(d_in, d_hidden)
        self.out = nn.Linear(d_hidden, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x[:, 1:]

        x = self.out(F.relu(self.first(x))).squeeze(2)
        return x

In [ ]:
device = "cuda"
batch_size = 16192
epochs = 100

d_token = 192
n_blocks = 3
attention_dropout = 0.2
ffn_dropout = 0.1
residual_dropout = 0.0
attention_heads = 8


reduction = "mean"

feature_tokenizer_kwargs = {
    "num_continous": len(X_train.columns.tolist()),
    "cat_cardinalities": (),
    "d_token": d_token,
}

dl_params = {
    "batch_size": batch_size,  # dataprallel splits batches across devices
    "shuffle": False,
    "device": device,
}

transformer_kwargs = {
    "d_token": d_token,
    "n_blocks": n_blocks,
    "attention_n_heads": attention_heads,
    "attention_initialization": "kaiming",
    "ffn_activation": ReGLU,
    "attention_normalization": nn.LayerNorm,
    "ffn_normalization": nn.LayerNorm,
    "ffn_dropout": ffn_dropout,
    # fix at 4/3, as activation (see search space B in
    # https://arxiv.org/pdf/2106.11959v2.pdf)
    # is static with ReGLU / GeGLU
    "ffn_d_hidden": int(d_token * (4 / 3)),
    "attention_dropout": attention_dropout,
    "residual_dropout": residual_dropout,  # see search space (B)
    "prenormalization": True,
    "first_prenormalization": False,
    "last_layer_query_idx": None,
    "n_tokens": None,
    "kv_compression_ratio": None,
    "kv_compression_sharing": None,
    "head_activation": nn.GELU, # nn.ReLU
    "head_normalization": nn.LayerNorm,
    "d_out": 1,  # fix at 1, due to binary classification
}

head_kwargs = {"d_in": d_token, "d_hidden": 32}

optim_params = {"lr": 1e-4, "weight_decay": 0.00001}

module_params = {
    "transformer": Transformer(**transformer_kwargs),  # type: ignore
    "feature_tokenizer": FeatureTokenizer(**feature_tokenizer_kwargs),  # type: ignore # noqa: E501
    "cat_features": None,
    "cat_cardinalities": [],
}

clf = FTTransformer(**module_params)
criterion = nn.BCEWithLogitsLoss()

# swap target head with classification head, and restore later
target_head = clf.transformer.head
clf_head = CLSHead(**head_kwargs)
clf.transformer.head = clf_head

In [ ]:
class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)

    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]

    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor

In [ ]:
optimizer = optim.AdamW(clf.parameters(),
    lr=optim_params["lr"],
    weight_decay=optim_params["weight_decay"],
)

max_iters = epochs * len(train_loader)
# saw recommendation of 5 - 10 % of total training budget or 100 to 500 steps
warmup = int(0.05 * max_iters)
print(f"warmup steps: {warmup}")
print(max_iters)

scheduler = CosineWarmupScheduler(optimizer=optimizer, warmup=warmup, max_iters=max_iters)

In [ ]:
def checkpoint(model):
    
    # remove old files
    for fn in glob.glob(f"checkpoints/{run.id}*"):
        os.remove(fn) 
    
    # create_dir
    dir_checkpoints = "checkpoints/"
    os.makedirs(dir_checkpoints, exist_ok = True) 
    
    # save new file
    print("saving new checkpoints.")
    torch.save(model.state_dict(), os.path.join(dir_checkpoints,f"{run.id}*"))

In [ ]:
# half precision, see https://pytorch.org/docs/stable/amp.html
scaler = torch.cuda.amp.GradScaler()


early_stopping = EarlyStopping(patience=15)

# see https://stackoverflow.com/a/53628783/5755604
# no sigmoid required; numerically more stable
# do not reduce, calculate mean after multiplication with weight

step = 0
best_accuracy = -1
best_step = -1

for epoch in tqdm(range(epochs)):

    # perform training
    loss_in_epoch_train = 0

    batch = 0
    
    for x_cat, x_cont, _, masks in train_loader:
    
        clf.train()
        optimizer.zero_grad()

        # for my implementation
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            logits = clf(x_cat, x_cont)
            train_loss = criterion(logits, masks)

        scaler.scale(train_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        scheduler.step()
        
        # add the mini-batch training loss to epoch loss
        loss_in_epoch_train += train_loss.item()
        
        wandb.log({"train_loss_step": train_loss.item(), "epoch": epoch, "batch": batch})


        batch += 1
        step +=1

    # clf.eval()
    # loss_in_epoch_val = 0.0
    # correct = 0
    
    # with torch.no_grad():
    #     for x_cat, x_cont, weights, targets in val_loader:

    #         # for my implementation
    #         logits = clf(x_cat, x_cont).flatten()
    #         logits = logits.flatten()

    #         val_loss = criterion(logits, targets)
            
    #         # get probabilities and round to nearest integer
    #         preds = torch.sigmoid(logits).round()
    #         correct += (preds == targets).sum().item()

    #         loss_in_epoch_val += val_loss  # val_loss #.item()
    #         wandb.log({"val_loss_step": val_loss, "epoch": epoch, "batch": batch})
            
    #         batch +=1      

    # loss average over all batches
    train_loss = loss_in_epoch_train / len(train_loader)
    # val_loss = loss_in_epoch_val / len(val_loader)
    
    # correct samples / no samples
    # val_accuracy = correct / len(X_val)
    # if best_accuracy < val_accuracy:
    #     checkpoint(clf, f"checkpoints/{run.id}-{step}.ptx")
    #     best_accuracy = val_accuracy
    #     best_step = step
    
    
    wandb.log({"train_loss": train_loss, 'epoch': epoch})
    # wandb.log({"val_loss": val_loss, 'epoch': epoch}) 
    
    # print(f"train:{train_loss} val:{val_loss}")
    # # print(f"val accuracy:{val_accuracy}")

    # return early if val accuracy doesn't improve. Minus to minimize.
    # early_stopping(-val_accuracy)
    # if early_stopping.early_stop or math.isnan(train_loss) or math.isnan(val_loss):
    #     print("meh... early stopping")
    #     break

        # hard_predictions = torch.zeros_like(predictions, dtype=torch.long)
        # hard_predictions[predictions > 0] = 1

        # # calculate column-wise accuracy
        # features_accuracy = (hard_predictions.bool() == masks).sum(0) / hard_predictions.shape[0]

        # mean_acc = features_accuracy.mean()

In [ ]:
# swap heads and save model
clf.transformer.head = target_head
checkpoint(clf)